In [ ]:
#IMPORT CONFIG FROM FILE .CONFIG
import configparser
import os
import sys
import re
from time import sleep
import database_functions as dbf
import update_prices_functions as upf
import telebot
from telebot import types
import pandas as pd
import numpy as np

config_file_name = 'price_monitoring.config'
#cwd = os.path.dirname(sys.argv[0]) #working on linux server or windows python script
cwd = os.getcwd() #working on windows during python notebook execution
cfg = configparser.ConfigParser()
ini_config_path = os.path.join(cwd,config_file_name)
cfg.read(ini_config_path)
database = cfg['database_config']['database']
bot_token = cfg['telegram_config']['bot_token']

link_prefix = cfg['general_config']['link_prefix']
link_sufix = cfg['general_config']['link_sufix']
link_sufix_split = cfg['general_config']['link_sufix_split']

#CREATE TELEGRAM BOT
bot = telebot.TeleBot(bot_token)

In [ ]:
#tem que ajustar o processo da virgula no target_price - ok
#criar como chave primaria o link, pois podem ter itens repetidos, na hora de cadastrar o produto - ok
#CRIAR ISSO COMO PRODUCT_ID E NAO COMO LINK - ok
#REPETIR O MESMO PARA A TABELA DE PRECOS - ok
#criar a lista de produtos cadastrados para servir de base para os comandos de consultar - ok
#criar as atividades de alterar preco - ok
#transformar o price_monitoring em um pacote - ok
#criar atividade que varre todos os produtos cadastrados em todos os DBs de usuarios e compilar novos precos, atualizando o db de precos - ok
#criar a atividade de excluir produto - ok
#ao cadastrar um produto novo, ja atualizar o db de precos - ok
#ajustar mensagem do produto cadastrado com sucesso
#ajustar a consulta dos itens cadastrados, para fazer o merge com a tabela de precos e mostrar o ultimo preco cadastrado - OK
#ajustar a lista de itens exibidos quando vai consultar o produto, ja mostrando a tabela mesclado com os precos
#criar atividade de enviar mensagem para o usuario quando atingir preco alvo - ok
#criar schedule para rodar a atividade de enviar mensagem para o usuario quando atingir preco alvo
#trocar as listas de consulta para inline keyboard



In [ ]:
## VALIDATING / OTHER FUNCTIONS
#EXTRACT URL FROM TEXT
def get_url(text):
    try:
        url = re.search(r'(https?://[^\s]+)', text).group(1)
    except:
        url = None
    return url

#VALIDATE NUMBER
def validate_number(text):
    try:
        text = re.search(r'(^\d+(.\d+)*$)', text).group(1)
    except:
        text = None
    return str(text)

# FUNCTION TO TRANSFORM SQLITE DB INTO DICTIONARY
def sqlite_table_to_dict(data, title):
    dict_data = []
    for row in data:
        dict_data.append(dict(zip(title, row)))
    return dict_data

In [ ]:
## BOT FUNCTIONS
user_dict = {}
class User:
    def __init__(self, chat_id):
        self.chat_id = chat_id
        self.url = None
        self.product_id = None
        self.product_link = None
        self.target_price = None

# HANDLE '/help' COMMAND
@bot.message_handler(commands=['help'])
def send_welcome(message):
    msg = bot.reply_to(message, """\
Ola, eu sou o bot de monitoramento de precos.
Envie o comando /start para iniciar.
""")

# HANDLE '/start' COMMAND
@bot.message_handler(commands=['start'])
def process_start_menu(message):
    msg_reply = "Ola, eu sou o bot de monitoramento de precos. O que deseja fazer?"
    try:
        markup = types.ReplyKeyboardMarkup(one_time_keyboard=True)
        markup.add('Cadastrar', 'Consultar', 'Sair')
        msg = bot.reply_to(message, msg_reply, reply_markup=markup)
        bot.register_next_step_handler(msg, process_start_menu_handler)
    except Exception as e:
        bot.reply_to(message, 'oooops, algo nao esta certo, tente novamente')

def process_start_menu_handler(message):
    try:
        answer = message.text
        if answer == u'Cadastrar':
            msg = bot.reply_to(message, 'Envie o link do produto que deseja monitorar')
            bot.register_next_step_handler(msg, process_url_step)
        elif answer == u'Consultar':
            chat_id = message.chat.id
            data, title = dbf.get_data_from_user_db(database, chat_id)
            dict_data = sqlite_table_to_dict(data, title)
            reply_list = '\n'.join([('Produto: ' + str(data['product_id']) + ' - Preco: ' + str(data['target_price'])) for data in dict_data])
            msg_reply = 'Aqui esta a sua lista de itens: \n\n' + reply_list + '\n\nInforme o que deseja realizar?'
            markup = types.ReplyKeyboardMarkup(one_time_keyboard=True)
            markup.add('Ajustar preco', 'Excluir item', 'Sair')
            msg = bot.reply_to(message, msg_reply, reply_markup=markup)
            bot.register_next_step_handler(msg, process_consult_step)
        elif answer == u'Sair':
            bot.reply_to(message, 'Obrigado, ate mais')
        else:
            raise Exception("Mensagem desconhecida")
    except Exception as e:
        bot.reply_to(message, 'oooops, algo nao esta certo, tente novamente')

def process_url_step(message):
    try:
        chat_id = message.chat.id
        user = User(chat_id)
        user_dict[chat_id] = user
        url = get_url(message.text)
        if url is not None:
            user.url = url
            product_id = url.split(link_prefix)[1].split(link_sufix_split)[0]
            user.product_id = product_id
            product_link = link_prefix + product_id + link_sufix
            user.product_link = product_link
            msg = bot.reply_to(message, 'Informe o preco alvo do produto: \n (Usar o ponto como separador decimal: 1000.60)')
            bot.register_next_step_handler(msg, process_target_price_step)
        else:
            msg = bot.reply_to(message, 'O link nao e valido. Informe novamente o link do produto:')
            bot.register_next_step_handler(msg, process_url_step)
    except Exception as e:
        bot.reply_to(message, 'oooops, algo nao esta certo, tente novamente')

def process_target_price_step(message):
    try:
        chat_id = message.chat.id
        target_price = message.text
        if validate_number(str(target_price)) is None:
            msg = bot.reply_to(message, 'O preco deve ser um numero. Utilize ponto para separar casas decimais.\n Informar novamente o preco alvo do produto:')
            bot.register_next_step_handler(msg, process_target_price_step)
            return
        user = user_dict[chat_id]
        user.target_price = validate_number(str(target_price))
        dbf.update_database(database, user.product_id, user.chat_id, user.product_link, user.target_price)
        msg_reply = 'Product ID: ' + str(user.product_id) + '\nURL: ' + user.product_link + '\nPreco alvo: ' + str(user.target_price)
        bot.send_message(chat_id, msg_reply)
        upf.register_price_from_url_list([user.product_link])
    except Exception as e:
        bot.reply_to(message, 'oooops, algo nao esta certo, tente novamente')

def process_consult_step(message):
    try:
        chat_id = message.chat.id
        answer = message.text
        product_id_list = dbf.read_all_product_id_from_user(database, str(chat_id))
        markup = types.ReplyKeyboardMarkup(one_time_keyboard=True)
        for product_id in product_id_list:
            markup.add(product_id)
        if answer == u'Ajustar preco':
            msg = bot.reply_to(message, 'Envie o ID do produto que deseja ajustar o preco', reply_markup=markup)
            bot.register_next_step_handler(msg, process_target_price_update_choose_item_step)
        elif answer == u'Excluir item':
            msg = bot.reply_to(message, 'Envie o ID do produto que deseja excluir', reply_markup=markup)
            bot.register_next_step_handler(msg, process_delete_choose_item_step)
        elif answer == u'Sair':
            bot.reply_to(message, 'Obrigado, ate mais')
        else:
            raise Exception("Mensagem desconhecida")
    except Exception as e:
        bot.reply_to(message, 'oooops, algo nao esta certo, tente novamente')

def process_target_price_update_choose_item_step(message):
    try:
        chat_id = message.chat.id
        user_dict['user'] = chat_id
        product_id = message.text
        user = user_dict['user']
        user_dict['product_id'] = product_id
        msg = bot.reply_to(message, 'Informe o preco alvo do produto: \n (Usar o ponto como separador decimal: 1000.60)')
        bot.register_next_step_handler(msg, process_target_price_update_step)
    except Exception as e:
        bot.reply_to(message, 'oooops, algo nao esta certo, tente novamente')

def process_target_price_update_step(message):
    try:
        chat_id = message.chat.id
        target_price = message.text
        product_id = user_dict['product_id']
        if validate_number(str(target_price)) is None:
            msg = bot.reply_to(message, 'O preco deve ser um numero. Utilize ponto para separar casas decimais.\nInformar novamente o preco alvo do produto:')
            bot.register_next_step_handler(msg, process_target_price_update_step)
            return
        user = user_dict['user']
        target_price = validate_number(str(target_price))
        product_id = user_dict['product_id']
        dbf.update_price_database(database, str(product_id), str(user), str(target_price))
        msg_reply = 'Product ID: ' + str(product_id) + '\nPreco alvo: ' + str(target_price) + '\nPreco atualizado com sucesso!'
        bot.send_message(chat_id, msg_reply)
    except Exception as e:
        bot.reply_to(message, 'oooops, algo nao esta certo, tente novamente')

def process_delete_choose_item_step(message):
    try:
        chat_id = message.chat.id
        user_dict['user'] = chat_id
        product_id = message.text
        user = user_dict['user']
        user_dict['product_id'] = product_id
        markup = types.ReplyKeyboardMarkup(one_time_keyboard=True)
        markup.add('Sim', 'Nao')
        msg = bot.reply_to(message, 'Tem certeza que deseja excluir o produto?', reply_markup=markup)
        bot.register_next_step_handler(msg, process_delete_step)
    except Exception as e:
        bot.reply_to(message, 'oooops, algo nao esta certo, tente novamente')

def process_delete_step(message):
    try:
        chat_id = message.chat.id
        answer = message.text
        user = user_dict['user']
        product_id = user_dict['product_id']
        if answer == u'Sim':
            dbf.delete_product_database(database, product_id, user)
            msg = bot.reply_to(message, 'O item {} foi excluido com sucesso!'.format(product_id))
        elif answer == u'Nao':
            msg = bot.reply_to(message, 'Ok, sem problemas. O item nao foi excluido.')
        else:
            raise Exception("Mensagem desconhecida")
    except Exception as e:
        bot.reply_to(message, 'oooops, algo nao esta certo, tente novamente')



In [ ]:
## BOT POLLING
# Enable saving next step handlers to file "./.handlers-saves/step.save".
# Delay=2 means that after any change in next step handlers (e.g. calling register_next_step_handler())
# saving will hapen after delay 2 seconds.
bot.enable_save_next_step_handlers(delay=2)

# Load next_step_handlers from save file (default "./.handlers-saves/step.save")
# WARNING It will work only if enable_save_next_step_handlers was called!
bot.load_next_step_handlers()

bot.infinity_polling()